In [31]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [32]:
import os
import pickle
import pandas as pd
import financial.data as fd
import financial.momentum.auxiliarFunctions as auxF
from financial.momentum.exponentialRegression import ExponentialRegressionModelFactory
from financial.io.file.cache import FileCache

# Configuración de prueba
ticker = "AAPL"
cache_path=os.environ["CACHE"]+"/"  # Ruta donde se guardarán los datos
model_name = "exponential"

# Crear el DataStore
ds = fd.CachedDataStore(path=os.environ.get("DATA", "./data"), cache=FileCache(cache_path=cache_path))

def local_regression_features_wrapper(ds: fd.DataStore) -> fd.Set:
    '''
    Wrapper function to generate local regression features.
    This is required by ModelFactory when creating a model.
    '''
    return auxF.local_regression_features(ds, ticker)

# Ejecutar la función para almacenar los datos
print(f"⚙️  Almacenando datos de momentum para {ticker}...")
auxF.store_exponentialModel_data(ticker, ds, cache_path)



⚙️  Almacenando datos de momentum para AAPL...


In [33]:
# Intentar recuperar los archivos guardados

prediction_path = slope_path = os.path.join(cache_path, f"model/momentum/{model_name}/{ticker}.pkl")
slope_path = os.path.join(cache_path, f"model/momentum/{model_name}/{ticker}@slope.pkl")
r2_path = os.path.join(cache_path, f"model/momentum/{model_name}/{ticker}@r2.pkl")

# Comprobar si los archivos existen
if os.path.exists(prediction_path) and os.path.exists(slope_path) and os.path.exists(r2_path):

    start_date = '1990-01-01'
    end_date = '2024-12-31'
    data = ds.get_data(ticker, start_date, end_date)

    print("\n📊 **Data Series")
    print(data)

    
    # Cargar los archivos y comprobar los datos
    with open(prediction_path, 'rb') as file:
        prediction_series = pickle.load(file)
        print("\n📊 **Prediction Series")
        print(prediction_series.apply(lambda x: f"{x:.15f}"))
    
    with open(slope_path, 'rb') as file:
        slope_series = pickle.load(file)
        print("\n📊 **Slope Series (Beta)")
        print(slope_series.apply(lambda x: f"{x:.15f}"))


    with open(r2_path, 'rb') as file:
        r2_series = pickle.load(file)
        print("\n📊 **R² Series")
        print(r2_series)

else:
    print("❌ ERROR: No se encontraron los archivos de caché.")


📊 **Data Series
Date
1990-01-02      0.262129
1990-01-03      0.263889
1990-01-04      0.264769
1990-01-05      0.265649
1990-01-08      0.267408
                 ...    
2024-12-24    258.200012
2024-12-26    259.019989
2024-12-27    255.589996
2024-12-30    252.199997
2024-12-31    250.419998
Name: AAPL, Length: 8817, dtype: float64

📊 **Prediction Series
Date
1990-05-10     6.397414219993095
1990-05-11     3.918637974062605
1990-05-14     6.955069819333374
1990-05-15     7.688544379450391
1990-05-16     8.753673791447511
                     ...        
2024-12-24    -4.235555274913793
2024-12-26    -3.979975755294490
2024-12-27    -2.120244519293067
2024-12-30    -0.304384714351116
2024-12-31     0.817062384635193
Length: 8727, dtype: object

📊 **Slope Series (Beta)
Date
1990-05-10    0.002618824292032
1990-05-11    0.002696137884735
1990-05-14    0.002798483610199
1990-05-15    0.002885968157494
1990-05-16    0.002974171705302
                    ...        
2024-12-24    0.00108